<a href="https://colab.research.google.com/github/sanmeshh/pytorch_learning/blob/dataset_loader_5/dataset_loader_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In the previous notebook we implemented batch gradient descent to train our module. but batch gd has problems like  memory insufficiency and slow convergence.

Therefore we will use mini batch gd in the below exercise

In [42]:
from sklearn.datasets import make_classification
import torch

In [43]:
X,y=make_classification(
    n_samples=10,
    n_features=2,
    n_informative=2,
    n_redundant=0,
    n_classes=2,
    random_state=42
)

In [44]:
X.shape

(10, 2)

In [45]:
X

array([[ 1.06833894, -0.97007347],
       [-1.14021544, -0.83879234],
       [-2.8953973 ,  1.97686236],
       [-0.72063436, -0.96059253],
       [-1.96287438, -0.99225135],
       [-0.9382051 , -0.54304815],
       [ 1.72725924, -1.18582677],
       [ 1.77736657,  1.51157598],
       [ 1.89969252,  0.83444483],
       [-0.58723065, -1.97171753]])

In [46]:
# Convert the data to Pytorch tensors
X=torch.tensor(X,dtype=torch.float32)
y=torch.tensor(y,dtype=torch.long)

In [47]:
y

tensor([1, 0, 0, 0, 0, 1, 1, 1, 1, 0])

In [48]:
from torch.utils .data import Dataset,DataLoader

In [49]:
class CustomDataset(Dataset):
  def __init__(self,features,labels):
    self.features=features
    self.labels=labels
   #dunder methods par parameters thoda hardcode kiya hai
  def __len__(self):
    return self.features.shape[0]

  def __getitem__(self,index):
    return self.features[index],self.labels[index]

In [50]:
dataset=CustomDataset(X,y)

In [51]:
len(dataset)

10

In [52]:
dataset[2]

(tensor([-2.8954,  1.9769]), tensor(0))

In [53]:
#iterable
dataloader=DataLoader(dataset,batch_size=2,shuffle=True)
#batch size = no of rows

In [54]:
for batch_features, batch_labels in dataloader:#randomaly selected since shuffle=True
  print(batch_features)
  print(batch_labels)
  print("-"*51)

tensor([[-0.9382, -0.5430],
        [-1.9629, -0.9923]])
tensor([1, 0])
---------------------------------------------------
tensor([[-0.5872, -1.9717],
        [-2.8954,  1.9769]])
tensor([0, 0])
---------------------------------------------------
tensor([[ 1.0683, -0.9701],
        [-1.1402, -0.8388]])
tensor([1, 0])
---------------------------------------------------
tensor([[-0.7206, -0.9606],
        [ 1.8997,  0.8344]])
tensor([0, 1])
---------------------------------------------------
tensor([[ 1.7273, -1.1858],
        [ 1.7774,  1.5116]])
tensor([1, 1])
---------------------------------------------------


# Sampler

In pytorch,the sampler in the DataLoader Class determines the strategy for selecting the samples from the dataset during data loading.

Pytorch provieds several predefined samplers,and you can create custom ones as well for eg you can create samples where we make sure in each batch there is atleast 1% of class 0.

1.Sequential Sampler

○ Samples elements sequentially, in the order they appear in the dataset.

○ Default when shuffle=False.



2.Random Sampler

○ Samples elements randomly without replacement.

○ Default when shuffle=True.


# Collate function

The collate_fn in PyTorch's DataLoader is a function that specifies how to combine a list of
samples from a dataset into a single batch. By default, the DataLoader uses a simple batch
collation mechanism, but collate_fn allows you to customize how the data should be
processed and batched(eg padding).



In [55]:
# Implementation


In [56]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [57]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [58]:
df.shape

(569, 33)

In [59]:
df.drop(columns=['id', 'Unnamed: 32'], inplace= True)

In [60]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


### train test split

In [61]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

### scaling

In [62]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [63]:
X_train

array([[-0.35782604, -0.72790385, -0.39621665, ..., -0.85372171,
         1.12184783, -0.7642126 ],
       [ 0.42422306,  0.9352511 ,  0.73239312, ...,  1.99063466,
         0.60723509,  1.03043961],
       [-0.54282691, -0.29724427, -0.56132958, ..., -0.39340098,
         0.51438447, -0.53012753],
       ...,
       [ 1.4865765 ,  0.01444858,  1.39365621, ...,  1.29162351,
         0.76775649,  0.31900459],
       [-1.08942037, -1.08480405, -1.05261154, ..., -1.11128391,
        -0.71942713,  0.05393767],
       [-0.58487256, -0.08786282, -0.62826725, ..., -0.95744107,
        -0.01753941, -0.17842619]])

In [64]:
y_train

,diagnosis
268,B
258,M
367,B
168,M
295,B
...,...
304,B
350,B
254,M
345,B


### Label Encoding

In [65]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [66]:
y_train

array([0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1,

### Numpy arrays to PyTorch tensors

In [67]:
X_train_tensor = torch.from_numpy(X_train.astype(np.float32))
X_test_tensor = torch.from_numpy(X_test.astype(np.float32))
y_train_tensor = torch.from_numpy(y_train.astype(np.float32))
y_test_tensor = torch.from_numpy(y_test.astype(np.float32))

In [68]:
X_train_tensor.shape

torch.Size([455, 30])

In [69]:
y_train_tensor.shape

torch.Size([455])

In [70]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):

  def __init__(self, features, labels):

    self.features = features
    self.labels = labels

  def __len__(self):

    return len(self.features)

  def __getitem__(self, idx):

    return self.features[idx], self.labels[idx]



In [71]:
train_dataset = CustomDataset(X_train_tensor, y_train_tensor)
test_dataset = CustomDataset(X_test_tensor, y_test_tensor)

In [72]:
train_dataset[10]

(tensor([-0.6690,  0.5546, -0.7070, -0.6405, -0.7069, -1.0523, -0.9104, -0.9674,
         -1.1562, -0.5784, -0.0150, -0.0174, -0.1263, -0.2641, -0.3932, -0.9030,
         -0.7445, -0.7433,  0.4289, -0.8094, -0.6819,  0.0225, -0.7226, -0.6396,
         -1.0240, -1.0908, -1.0440, -1.1655, -0.7210, -1.0505]),
 tensor(0.))

In [73]:
#we will load our batches of xtrain/xtest.. with the help of DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

### Defining the model

In [74]:
import torch.nn as nn


class MySimpleNN(nn.Module):

  def __init__(self, num_features):

    super().__init__()
    self.linear = nn.Linear(num_features, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, features):

    out = self.linear(features)
    out = self.sigmoid(out)

    return out

### Important Parameters

In [75]:
learning_rate = 0.1
epochs = 25

In [76]:
# create model
model = MySimpleNN(X_train_tensor.shape[1])

# define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# define loss function
loss_function = nn.BCELoss()

### Training Pipeline

In [77]:

# define loop
for epoch in range(epochs):
  #to load batches
  for batch_features, batch_labels in train_loader:

    # forward pass
    y_pred = model(batch_features)

    # loss calculate              #reshaping the labels in the batch
    loss = loss_function(y_pred, batch_labels.view(-1,1))

    # clear gradients



    # backward pass
    loss.backward()

    # parameters update
    optimizer.step()

  # print loss in each epoch
  print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

Epoch: 1, Loss: 0.0006681004888378084
Epoch: 2, Loss: 7.470060836567427e-07
Epoch: 3, Loss: 0.05301233381032944
Epoch: 4, Loss: 5.522728585560799e-08
Epoch: 5, Loss: 2.0318901538848877
Epoch: 6, Loss: 1.8199858686784864e-06
Epoch: 7, Loss: 1.2929526782556877e-08
Epoch: 8, Loss: 9.375750164395857e-29
Epoch: 9, Loss: 2.4462995753259475e-26
Epoch: 10, Loss: 0.0009360041585750878
Epoch: 11, Loss: 0.0
Epoch: 12, Loss: 2.0342159296291045e-16
Epoch: 13, Loss: 0.000873793731443584
Epoch: 14, Loss: 1.5635961294174194
Epoch: 15, Loss: 0.0
Epoch: 16, Loss: 6.898895740509033
Epoch: 17, Loss: 0.0
Epoch: 18, Loss: 0.005591458175331354
Epoch: 19, Loss: 0.0
Epoch: 20, Loss: 0.0
Epoch: 21, Loss: 1.2613739935216513e-23
Epoch: 22, Loss: 0.0
Epoch: 23, Loss: 0.0
Epoch: 24, Loss: 9.083074570259697e-11
Epoch: 25, Loss: 0.0


In [78]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [79]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [80]:
df.shape

(569, 33)

In [81]:
df.drop(columns=['id', 'Unnamed: 32'], inplace= True)

In [82]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


### train test split

In [83]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

### scaling

In [84]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [85]:
X_train

array([[-9.24340117e-03,  4.99813510e-02, -1.84927157e-03, ...,
         1.06904186e+00,  4.46842296e-01,  1.15604912e+00],
       [-4.85399038e-01,  5.69107027e-01, -4.86444829e-01, ...,
         1.00701728e-01,  1.07152707e-01,  4.51265830e-01],
       [-1.03797471e+00,  1.07132802e-01, -1.04454165e+00, ...,
        -7.26590791e-01, -3.88974063e-02, -3.75993372e-01],
       ...,
       [ 1.17526723e+00,  2.73824533e-01,  1.16391500e+00, ...,
         5.42837738e-01,  5.64995196e-01, -4.43245773e-01],
       [ 1.78662755e+00,  1.05965698e+00,  1.86046769e+00, ...,
         1.21849629e+00,  6.35558734e-01,  5.64993467e-01],
       [-8.61620775e-01, -1.38142174e-01, -7.63356576e-01, ...,
         2.15414327e+00,  2.04682949e+00,  3.05879996e+00]])

In [86]:
y_train

,diagnosis
435,M
88,B
245,B
28,M
393,M
...,...
377,B
294,B
492,M
302,M


### Label Encoding

In [87]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [88]:
y_train

array([1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0,
       1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,

### Numpy arrays to PyTorch tensors

In [89]:
X_train_tensor = torch.from_numpy(X_train.astype(np.float32))
X_test_tensor = torch.from_numpy(X_test.astype(np.float32))
y_train_tensor = torch.from_numpy(y_train.astype(np.float32))
y_test_tensor = torch.from_numpy(y_test.astype(np.float32))

In [90]:
X_train_tensor.shape

torch.Size([455, 30])

In [91]:
y_train_tensor.shape

torch.Size([455])

In [92]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):

  def __init__(self, features, labels):

    self.features = features
    self.labels = labels

  def __len__(self):

    return len(self.features)

  def __getitem__(self, idx):

    return self.features[idx], self.labels[idx]



In [93]:
train_dataset = CustomDataset(X_train_tensor, y_train_tensor)
test_dataset = CustomDataset(X_test_tensor, y_test_tensor)

In [94]:
train_dataset[10]

(tensor([ 0.1671, -1.3693,  0.1336,  0.0492,  0.1383, -0.2842, -0.0599, -0.1173,
         -0.2583, -0.9149,  0.0702, -1.0921, -0.1374, -0.0461, -0.7078, -0.4147,
         -0.0556, -0.5086, -0.6385, -0.5735,  0.1257, -1.4201,  0.0615, -0.0119,
         -0.4530, -0.3970, -0.1085, -0.3452, -0.4606, -0.7587]),
 tensor(0.))

In [95]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

### Defining the model

In [96]:
import torch.nn as nn


class MySimpleNN(nn.Module):

  def __init__(self, num_features):

    super().__init__()
    self.linear = nn.Linear(num_features, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, features):

    out = self.linear(features)
    out = self.sigmoid(out)

    return out

### Important Parameters

In [97]:
learning_rate = 0.1
epochs = 25

In [98]:
# create model
model = MySimpleNN(X_train_tensor.shape[1])

# define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# define loss function
loss_function = nn.BCELoss()

### Training Pipeline

In [99]:

# define loop
for epoch in range(epochs):

  for batch_features, batch_labels in train_loader:

    # forward pass
    y_pred = model(batch_features)

    # loss calculate
    loss = loss_function(y_pred, batch_labels.view(-1,1))

    # clear gradients, so that the gradients dont add up
    optimizer.zero_grad()

    # backward pass
    loss.backward()

    # parameters update
    optimizer.step()

  # print loss in each epoch
  print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

Epoch: 1, Loss: 0.335091769695282
Epoch: 2, Loss: 0.19809426367282867
Epoch: 3, Loss: 0.2799932658672333
Epoch: 4, Loss: 0.055179301649332047
Epoch: 5, Loss: 0.10304231941699982
Epoch: 6, Loss: 0.2676098644733429
Epoch: 7, Loss: 0.07065895199775696
Epoch: 8, Loss: 0.028295952826738358
Epoch: 9, Loss: 0.35768136382102966
Epoch: 10, Loss: 0.04688691720366478
Epoch: 11, Loss: 0.006624013651162386
Epoch: 12, Loss: 0.614785373210907
Epoch: 13, Loss: 0.1140715479850769
Epoch: 14, Loss: 0.14232571423053741
Epoch: 15, Loss: 0.06242777779698372
Epoch: 16, Loss: 0.0109254764392972
Epoch: 17, Loss: 0.09088870137929916
Epoch: 18, Loss: 0.03906431794166565
Epoch: 19, Loss: 0.010040667839348316
Epoch: 20, Loss: 0.10085105150938034
Epoch: 21, Loss: 0.05963810533285141
Epoch: 22, Loss: 0.07073384523391724
Epoch: 23, Loss: 0.07617320865392685
Epoch: 24, Loss: 0.08514343202114105
Epoch: 25, Loss: 0.20222432911396027


In [107]:
model.eval()
accuracy_list=[]

with torch.no_grad():
  #calculating batch accuracies
  for batch_features, batch_labels in test_loader:
    y_pred=model(batch_features)
    y_pred=(y_pred>0.8).float() #converting probalilties into binary predictions

    batch_accuracy=(y_pred.view(-1)==batch_labels).float().mean().item()

    accuracy_list.append(batch_accuracy)


#taking the average accuracy of batches
overall_acc=sum(accuracy_list)/len(accuracy_list)
print(f'Accuracy:{overall_acc:.4f}')





Accuracy:0.9609
